### collect

collect raw data from a folder by date  
a folder can be a local backup or online; depends on mobile device store  
raw data is given by a special mobile app (developed for this project)  
  
every folder has a list of image files with a corresponding meta file  
meta files containing sample specific image information about 
- camera settings, 
- user inputs and 
- on site conditions (weather, location)

In [10]:
import os
import numpy as np
import configparser as cp

import datetime, socket

fpath = os.path.join(os.getcwd(), "raw", "2023-06-14")
fmeta = [[f,[os.path.join(os.path.join(fpath,f),m) for m in os.listdir(os.path.join(fpath,f)) if m.endswith(".meta")]] for f in os.listdir(fpath) if os.path.isdir(os.path.join(fpath,f))]
fnames = [(m[0],m[1][0]) for m in fmeta if len(m[1])>0]

# stamp of sample (known)
sdate = datetime.datetime.strptime("2023-06-14","%Y-%m-%d").date()

def ini_date(fname):
  rc = None
  conf = None
  try:
      conf = cp.ConfigParser(interpolation=None)
      conf.read(fname, encoding="utf8")
      rc = datetime.datetime.strptime(conf["image"]["stamp"],"%Y-%m-%d %H:%M:%S").date()            
  except cp.ParsingError as argument:            
      conf.clear()
  return rc
  
ffolders = [(fid,ini_date(fname),fname) for fid,fname in fnames] 
ffolders = [[fid,fname] for fid,fdate,fname in ffolders if fdate==sdate]

print(f"folders: {len(ffolders)}, files per folder: {len(fnames)}, sample date: {sdate}")

print(f"author  {socket.gethostname():>28}")
print(f"version {str(datetime.datetime.now()):>28}")


folders: 15, files per folder: 28, sample date: 2023-06-14
author                           NOB
version   2023-08-20 03:14:03.773152


### request   

request meta data for every folder by a given sample concentration, reagent and their timestamp  

In [8]:
def ini_meta(fname):
  rc = None
  conf = None
  try:
      
      conf = cp.ConfigParser(interpolation=None)
      conf.read(fname, encoding="utf8")

      con = conf["controls"]["concentration"]
      for s,t in [(">",""),(",","."),("?","0")]: con = con.replace(s,t)
      rea = conf["controls"]["reagent"]
      dat = datetime.datetime.strptime(conf["image"]["stamp"],"%Y-%m-%d %H:%M:%S").date()

      flo = np.round(float(con),6)
      rc = [rea,flo,str(flo),dat]

  except cp.ParsingError as argument:            
      conf.clear()
  return rc

fmeta = [ini_meta(fname)+[fid,fname] for fid,fname in ffolders]

In [9]:
import pandas as pd
fframes = pd.DataFrame(fmeta,columns=["rea","flo","slo","dat","id","info"]).sort_values(by=["rea","flo"]).reset_index(drop=True)

### copy

copy files from raw file folders to data folders  
raw file folders data is given by uuid/\*.\*  
data file folders files then ordered by <b>data\\stamp\\reagent</b>  
  
> this gives a label for further analysis  

In [10]:
import shutil

fdata = os.path.join(os.getcwd(),"data")

for index,row in fframes.iterrows():

  fsource = os.path.join(fpath,row["id"])  
  ftarget = os.path.join(fdata,str(row["dat"]),row["rea"],row["slo"])
  finfo = row["info"]
  
  if not os.path.exists(ftarget): os.makedirs(ftarget)  
  
  fnames = [os.path.join(fsource,fname) for fname in os.listdir(fsource) if os.path.isfile(os.path.join(fsource,fname)) & fname.endswith(".jpg")]

  if len(fnames)==0:
    continue
  
  shutil.copy2(finfo,ftarget)

  for fname in fnames: 
    shutil.copy2(fname,ftarget)

last (manual) update: 2023-08-28